# Neural ODEs
Red Neuronal para simulacion de EDOs de primer ordel del tipo
$$\dfrac{d}{dt}S(t)=P_1(t) + P_2(t)*P_3(S)$$
donde $P_1$, $P_2$ y $P_3$ son polinomios.

In [94]:
# Recarga los modulos
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [95]:
import os

In [96]:
import warnings
warnings.filterwarnings('ignore')

In [97]:
import random

In [98]:
import gpustat
gpustat.print_gpustat()

Lucifer                                2022-12-02 19:21:59  526.98
[0] NVIDIA GeForce RTX 3080 Laptop GPU | 52°C,   0 % |  1331 /  8192 MB | LUCIFER\Det-Pc(?M) LUCIFER\Det-Pc(?M)


In [99]:
import numpy as mp
import pandas as pd

In [100]:
import matplotlib.pyplot as plt
%matplotlib inline

In [101]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data as data_torch

In [102]:
from tqdm import tnrange, tqdm_notebook

In [103]:
from api.utils import *

## Tratamiento de datos

In [104]:
# Obtenemos datos de las simulaciones
file_name = 'datos_ODEs_cos_100.csv'
dir_file = 'C:/Users/Det-Pc/OneDrive/Documentos/GitHub/Proyecto_curso_DL/data'
data = get_ode_data(dir_file + '/' + file_name)
# data.head()

El tamano de los datos de polinomio en S y t es: 0
El tamano de los datos de polinomio en S es    : 0
El tamano de los datos de polinomio en t es    : 0
El numero total de los datos es                : 999993


In [105]:
data.head()

,funcion,grade_f _t,grade_f_s,I_c,n_steps,int_0,int_f,sol.t,sol.y
0,cos_s_t,0,0,0.010087,100,0,1,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.010087274744655872, 0.010163938257431315, 0..."
1,cos_s_t,0,0,1.462783,100,0,1,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[1.4627829216984938, 1.4627793823005937, 1.462..."
2,cos_s_t,0,0,1.617596,100,0,1,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[1.6175964330144468, 1.6345681291574026, 1.651..."
3,cos_s_t,0,0,-0.537306,100,0,1,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[-0.5373064398304225, -0.5373251786954568, -0...."
4,cos_s_t,0,0,-0.683805,100,0,1,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[-0.6838045572046187, -0.6693757742090535, -0...."


In [106]:
# Extraemos los valores de las simulaciones
df_values = data['sol.y'].values

### DataSets

In [107]:
seq_len = 75
size_predic = len(data['sol.y'].values[0])
len_sim = df_values.shape[0]

In [108]:
# Creamos las sucesiones con los datos estandarizados
X_data, Y_data = get_seqs(data=df_values, seq_len=seq_len)

In [109]:
print(f'EL tamano de los datos las sucesiones_X es: {X_data.shape}')
print(f'EL tamano de los datos las sucesiones_Y es: {Y_data.shape}')

EL tamano de los datos las sucesiones_X es: (999918, 75)
EL tamano de los datos las sucesiones_Y es: (999918, 25)


In [110]:
# porcentaje de datos de entrenamiento
pct_data_train = 0.8

In [111]:
index_data = list(range(X_data.shape[0]))

train_index = random.sample(index_data, int(len(index_data)*pct_data_train))
val_test_index = list(set(index_data)-set(train_index))
val_index = random.sample(val_test_index, int(len(val_test_index)*0.5))
test_index = list(set(val_test_index)-set(val_index))

In [112]:
# Creamos las particiones
data_dic = {'train':X_data[train_index],\
           'val':X_data[val_index],\
           'test':X_data[test_index]}

In [113]:
class EDOs_dataloader(data_torch.Dataset):
    def __init__(self, edos_s):
        self.edos_s = edos_s
        
    def __getitem__(self, index):
        edo_i = self.edos_s[index]
        return edo_i
        
    def __len__(self):
        return len(self.edos_s)

In [114]:
partitions = ['train', 'val', 'test']
batch_size = {'train': 512, 'val':256, 'test':1}
shuffle = True
lr = 0.001
edos_datasets = {x: EDOs_dataloader(data_dic[x]) for x in partitions}
dataloaders = {x: torch.utils.data.DataLoader(edos_datasets[x], batch_size=batch_size[x], shuffle=shuffle) for x in partitions}
dataset_sizes = {x: len(edos_datasets[x]) for x in partitions}
print('Se cargo la data exitosamente!')
print(f'El tamano de la dataset es {dataset_sizes}')

Se cargo la data exitosamente!
El tamano de la dataset es {'train': 799934, 'val': 99992, 'test': 99992}


## Modelo

In [115]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Estamos usando: {}'.format(device))

Estamos usando: cuda


In [116]:
import torch
from torch.autograd import Variable

import torch.nn as nn


class TimeDistributed(nn.Module):
    def __init__(self, module, batch_first=True):
        # def __init__(self, module, batch_first=False):
        super(TimeDistributed, self).__init__()
        self.module = module
        self.batch_first = batch_first

    def forward(self, x):

        if len(x.size()) <= 2:
            return self.module(x)

        # concatena samples y timesteps en un solo vector
        x_reshape = x.contiguous().view(-1, x.size(-1))  # (samples * timesteps, input_size)

        y = self.module(x_reshape)

        # We have to reshape Y
        if self.batch_first:
            y = y.contiguous().view(x.size(0), -1, y.size(-1))  # (samples, timesteps, output_size)
        else:
            y = y.view(-1, x.size(1), y.size(-1))  # (timesteps, samples, output_size)

        return y

class ODE_NET(nn.Module):
    def __init__(self, hidden_dim):
        super(ODE_NET, self).__init__()
        # self.hidden_dim = hidden_dim
        self.conv1 = nn.Conv1d(in_channels=1,out_channels=32,kernel_size=6,stride=2,padding=3)
        self.relu = nn.ReLU()
        # self.pool = nn.MaxPool1d(2, 2)
        self.conv2 = nn.Conv1d(in_channels=32,out_channels=16,kernel_size=6,stride=2,padding=3)
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # self.fc2 = nn.Linear(120, 84)
        # self.fc3 = nn.Linear(84, 10)
        self.rnn1 = nn.LSTM(input_size=640, hidden_size=64, num_layers=1, batch_first=True)
        self.linear = nn.Linear(64, size_predic-seq_len)

    def forward(self, x, hidden):
        print(x.shape)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        print(x.shape)
        x = x.view(-1,)
        print(x.shape[0])
        x, hidden = self.rnn1(x, hidden)
        x = x.contiguous().view(-1, self.hidden_dim)
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        x = self.linear(x)
        return x, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                    weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden


In [117]:
class Model_S(nn.Module):
    def __init__(self, seq_len):
        super(Model_S, self).__init__()
        self.seq_len = seq_len
        # self.flatten = nn.Flatten()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=12, kernel_size=6, stride=2, padding=3)
        self.pool1 = nn.MaxPool1d(2, 2)
        self.conv2 = nn.Conv1d(in_channels=26, out_channels=14, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool1d(2, 2)
    
    def forward(self, x):
        print('El tamano de la entrada es: ', x.shape)
        # El tamano de la entrada es:  torch.Size([2, 1, 75])

        # x = self.flatten(x)
        # print('El tamano despues de flatten es: ', x.shape)
        
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        print('El tamano despues de la conv1 es: ', x.shape)
        # El tamano despues de la conv1 es:  torch.Size([2, 12, 19])

        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        print('El tamano despues de la conv2 es: ', x.shape)
        

        return x

In [118]:
model = Model_S(seq_len=seq_len)
model.to(device)

Model_S(
  (conv1): Conv1d(1, 12, kernel_size=(6,), stride=(2,), padding=(3,))
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(26, 14, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [119]:
# model = ODE_NET(hidden_dim=75)
# model.to(device)

In [120]:
from torchsummary import summary
summary(model, (1,seq_len))

El tamano de la entrada es:  torch.Size([2, 1, 75])
El tamano despues de la conv1 es:  torch.Size([2, 12, 19])


RuntimeError: Failed to run torchsummary. See above stack traces for more details. Executed layers up to: [Conv1d: 1-1, MaxPool1d: 1-2]